# **응급상황 자동 인식 및 응급실 연계 서비스**

### (1) 미션1
* 음성인식 : STT(Speech-to-Text)
    * 사용 모델 : OpenAI의 **Whisper-1**
    * 제공받은 음성 파일과 새로 제작하는 5건 이상의 음성파일을 텍스트로 변환하고, 변환작업이 잘 되는지 확인해 봅시다.

### (2) 미션2
* 텍스트 요약 및 핵심 키워드 도출
    * 사용 모델 : OpenAI의 **GPT-3.5-turbo**
    * 내용 요약과 주요 키워드를 도출하도록
    프롬프트 입력과 출력을 구성하고 테스트 해 봅시다.

* [추가]응급실 현황 다운로드(이 데이터는 단계3에서 필요합니다.)



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
path = '/content/drive/MyDrive/project6/2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6/2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json

# 더 필요한 라이브러리 추가 -------------




### (3) OpenAI API Key 환경 변수 설정

* 제공받은 open ai api key를 **api_key.txt** 파일에 저장합니다.
    * (제공받은 api_key.txt 파일은 비어 있습니다.)

* 다음 코드를 통해 환경변수로 등록 합니다.

In [ ]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 결과는 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2.  STT**

### (1) 제공된 데이터 변환
* 세부사항
    * 사용 모델 : whisper-1
    * 제공 받은 오디오 파일을 읽어서 텍스트로 변환시켜 봅시다.
        * 반복문을 통해 파일 하나씩 읽어서 텍스트 변환
        * 변환된 텍스트를 데이터 프레임에 추가

|filename|text|
|----|----|
|audio3.mp3|어쩌구 저쩌구...급해요.|

* 음성파일 변환

In [ ]:
# 음성파일 경로 지정
audio_path = '/content/drive/MyDrive/project6/2/audio/'

In [ ]:
# OpenAI 클라이언트 생성
client = OpenAI()

In [ ]:
# 위스퍼 모델 사용 : 제공된 음성파일 중 1개를 텍스트로 변환해보기
filename = 'audio2.mp3'
audio_file = open(audio_path + filename, "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
)

print(transcript, type(transcript))

119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?
 <class 'str'>


* 음성파일 변환 함수 생성

In [ ]:
def audio_to_text(audio_path, filename):
    client = OpenAI()

    # 오디오 파일을 읽어서, 위스퍼를 사용한 변환
    with open(audio_path + filename, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            language="ko",
            response_format="text",
        )

    # transcript 확인
    # print(transcript, type(transcript))

    # transcript에서 개행 문자 제거
    transcript = transcript.strip()  # 양쪽 끝의 공백 및 개행 문자 제거

    return transcript

In [ ]:
audio_to_text(audio_path, 'audio2.mp3')

'119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?'

In [ ]:
# 음성파일 이름을 리스트에 담기
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]
print(file_names)

['audio1.mp3', 'audio5.mp3', 'audio3.mp3', 'audio2.mp3', 'audio4.mp3', 'audio06.mp3', 'audio07.mp3', 'audio10.mp3', 'audio09.mp3', 'audio11.mp3', 'audio08.mp3', 'audio12.mp3', 'audio14.mp3', 'audio13.mp3', 'audio15.mp3']


In [ ]:
# 반복문을 통해, 파일 하나씩 읽어서 텍스트 변환, 변환된 텍스트를 데이터 프레임에 추가

# 빈 데이터프레임 선언
audio_df = pd.DataFrame(columns=["filename", "transcript"])

# 반복문 수행하면서 오디오 변환
for filename in file_names:
    try:
        transcript = audio_to_text(audio_path, filename)
        # 데이터프레임에 추가
        audio_df = pd.concat([audio_df, pd.DataFrame({"filename": [filename], "transcript": [transcript]})], ignore_index=True)
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# 데이터프레임 결과 조회
print(audio_df)


       filename  \
0    audio1.mp3   
1    audio5.mp3   
2    audio3.mp3   
3    audio2.mp3   
4    audio4.mp3   
5   audio06.mp3   
6   audio07.mp3   
7   audio10.mp3   
8   audio09.mp3   
9   audio11.mp3   
10  audio08.mp3   
11  audio12.mp3   
12  audio14.mp3   
13  audio13.mp3   
14  audio15.mp3   

                                                                                                                                                                                                                                     transcript  
0                                                                                지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.  
1                                                                                                                                                                             화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아

### (2) 오디오 데이터 추가 수집(제작) 및 변환

* 세부사항
    * 응급 상황에 맞는 음성 녹음하기
        * 응급 등급별 1개 이상씩(총 5개 이상)
    * 반복문을 통해 모든 음성 파일 데이터 변환 : STT
        * 변환 내용은 위에서 저장한 데이터프레임에 추가
    * 변환 후 음성 내용과 변환 결과를 비교


## **3. Summary**

* 세부사항
    * 문서요약 예제 파일을 참조하여 테스트 해 봅니다.
    * 코드를 참조하여, 원하는 형식에 맞게 요약이 되도록 프롬프트를 구성합니다.
        * 요약 시 중요 키워드들이 함께 도출되도록 합니다.
        * 가능하다면, 요약 문장 길이에 제한을 둡시다.
    * 반복문을 통해 요약하고, 결과를 데이터프레임에 추가합니다.
        * summary 열을 추가하고, 요약 결과를 입력
            * 요약결과와 키워드는 하나의 문자열로 붙여서 summary열에 추가

### (1) 문서 요약

* 문서 요약 예제

In [ ]:
input_text = '''
한국은행 총재가 "올해 성장률이 기존 전망치 2.4%보다 낮아질 가능성이 크다"며 "2.2∼2.3% 정도로 떨어지지 않을까 생각한다"고 밝혔습니다.
이 총재는 오늘(29일) 국회 기획재정위원회 국정감사에 출석해 한은의 전망을 크게 밑돈 3분기 성장률을 바탕으로 올해 성장률 전망치가 조정될 가능성에 대해 이렇게 말했습니다.
성장률 하락의 가장 큰 요인인 수출 감소의 배경에 대해 이 총재는 "금액 기준으로 봐서는 수출이 안 떨어졌는데, 수량을 기준으로 떨어졌다"며 "자동차 파업 등 일시적 요인과 화학제품·반도체의 중국과 경쟁 등으로 수량이 안 늘어나는 것 같은데, 원인을 더 분석해봐야 할 사안"이라고 진단했습니다.
다음 달 28일 열릴 기준금리 결정 방향에 대해서는 "금리 결정할 때 하나의 변수만 보지 않고 종합적으로 보는데, 우선 미국 대선과 연방준비제도 금리 결정으로 경제 상황이 어떻게 변할지 보겠다"고 밝혔습니다.
또 "아울러 이후 달러가 어떻게 될지, 수출 등 내년 경제 전망과 거시안전성 정책이 부동산·가계부채에 미치는 영향 등도 고려해 결정하겠다"고 말했습니다.
'''

system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
응답은 다음의 형식을 지켜주세요
{"summary": \"텍스트 요약\"}
'''

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": input_text
        }
    ]
)

# 답변
answer = response.choices[0].message.content
print(answer)

{"summary": "한국은행 총재는 올해 성장률이 기존 전망치보다 낮아질 가능성이 크다고 밝히며 2.2∼2.3%로 전망치가 조정될 것으로 예상했다. 수출 감소가 주요 이유로 손을 든 배경을 분석하며 미국 대선, 달러 가격 변동 등을 고려하여 다음 달의 기준금리 결정 방향을 결정할 것이라고 밝혔다."}


* 문서 요약 함수로 생성

In [ ]:
def text_summary(input_text):
    # OpenAI 클라이언트 생성


    # 시스템 역할과 응답 형식 지정




    # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답 받기


    # 응답형식을 정리하고 return



In [ ]:
def text_summary(input_text):
    # OpenAI 클라이언트 생성
    client = OpenAI()

#  요약 결과로 KTAS 5등급에 따라 응급실 방문 여부를 판단해서 요약 내용에 붙여주세요.
    # 시스템 역할과 응답 형식 지정
    system_role = '''당신은 응급실에서 신고를 요약해서 전달하는 간호사입니다.
    응답은 다음의 형식을 지켜주세요: {"summary": "텍스트 요약", "keywords": ["키워드1", "키워드2", ...]}
    환자의 인적 정보와 증상을 육하원칙에 따라 정리해서 응답해야 합니다. 총 길이는 60자 이내입니다.
    예를 들면 * 중요 키워드 : 우도, 등산객, 상처, 출혈, 의식 흐림, 응급차, * 내용 요약 : 우도 등산객, 다친 깊은 상처 출혈. 의식 흐림, 지속 출혈. 응급차 요청 중.
    * 중요 키워드 : 아버지, 마흔아홉, 넘어짐, 피출혈, 어지러움, 응급실, * 내용 요약 : 마흔아홉 살 아버지가 계단에서 넘어져 머리에서 피가 나며 어지러움을 호소 중. 응급실 방문 고려 요망.
    이런식으로 결과가 나와야 합니다.'''

    # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답 받기
    answer = response.choices[0].message.content

    # 응답 형식 정리
    # 예: {"summary": "요약문", "keywords": ["키워드1", "키워드2"]}
    result = eval(answer)  # JSON 형식으로 변환
    # result = json.loads(answer)
    return result['summary'], result['keywords']

* 저장된 text를 하나씩 불러와서 요약하고 다시 저장하기

In [ ]:
# summary 열 추가
audio_df['summary'] = None  # 요약 결과를 저장할 열 생성

# 각 transcript를 요약하여 summary 열에 추가
for index, row in audio_df.iterrows():
    transcript = row['transcript']
    summary, keywords = text_summary(transcript)
    # 형식 변경: 중요 키워드 : keywords, 내용 요약 : summary
    summary_with_keywords = f"* 중요 키워드 : {', '.join(keywords)}, * 내용 요약 : {summary}"
    audio_df.at[index, 'summary'] = summary_with_keywords


In [ ]:
# 결과 출력
audio_df

,filename,transcript,summary
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.,"* 중요 키워드 : 아버지, 마흔아홉, 넘어짐, 피출혈, 어지러움, 응급실, * 내용 요약 : 마흔아홉 살 아버지가 계단에서 넘어져 머리 피출혈로 어지러움 호소 중. 응급실 방문 고려 요망."
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈 것 같은데 응급실을 가야 할까요?,"* 중요 키워드 : 미끄러짐, 엉덩방아, 꼬리뼈, 아픔, 응급실, * 내용 요약 : 화장실에서 미끄러져 엉덩방아 찍고 꼬리뼈 계속 아픔. 응급실 방문 고려 요망"
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안 빠져요. 어떻게 해야 할까요? 동생이 너무 힘들어 하네요.,"* 중요 키워드 : 콩, 코막힘, 동생, 응급실, * 내용 요약 : 콩으로 한쪽 코 막힘. 제거 불가. 응급실 방문 고려."
3,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?,"* 중요 키워드 : 38도, 머리 통증, 어지러움, 열, 수액, * 내용 요약 : 38도 열이 나며 머리 통증, 어지러움 호소 중. 응급 상황 아님. 쉬면서 수액 섭취 권고."
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리가 어지럽고 속이 매스꺼워요. 어떻게 해야 할까요?","* 중요 키워드 : 머리 박음, 어지러움, 속 메스꺼움, 응급실, * 내용 요약 : 머리를 박은 후 3시간이 지나 머리 어지럽고 속 메스꺼움. 응급실 방문 권장됨."
5,audio06.mp3,갑자기 심각한 알레르기 반응이 왔어요 저희 신랑이 갑자기 두드러기와 호흡 곤란을 보이고 있어요 얼굴과 입술이 부풀어 오르고 있고 숨을 쉬기 힘들어 합니다 저희 집 주소는 영종도 온서역 근처이며 알레르기 원인은 오이 입니다 도와주세요,"* 중요 키워드 : 알레르기 반응, 호흡 곤란, 두드러기, 오이, 영종도, 온서역, * 내용 요약 : 신랑이 오이에 대한 심각한 알레르기 반응, 두드러기, 호흡 곤란, 부어오른 얼굴과 입술, 온서역 주변. 응급 신고 중."
6,audio07.mp3,여기는 분당로 42번 길입니다. 지금 교통사고가 발생했습니다. 빨간옷을 입은 여자가 차에 치여서 심하게 다쳤고 다리에서 대량으로 출혈이 나고 있습니다. 의식이 없는 것 같고 심각한 상태입니다. 출혈이 멈추지 않고 있어요. 어떻게 해야 할지 알려주세요.,"* 중요 키워드 : 분당로 42번 길, 교통사고, 빨간옷 여자, 출혈, 의식 없음, * 내용 요약 : 길에서 빨간옷 여자 차에 치여 다쳐 출혈 중인 상황. 의식 없음. 출혈 멈추지 않음."
7,audio10.mp3,안녕하세요. 지금 전철 안에 50대 여성이 심각한 증상을 보이고 있습니다. 고열이 나고 기침과 가래가 심하게 나오고 있어요. 체온은 39도 이상이고 숨쉬는 것도 힘들어합니다. 저희 위치는 5호선 구분다리역입니다. 제발 즉시 응급차를 보내주세요.,"* 중요 키워드 : 5호선, 고열, 호흡곤란, 50대 여성, 응급차, * 내용 요약 : 5호선 구분다리역에서 고열과 호흡곤란을 호소하는 50대 여성 발견. 긴급 응급차 요청"
8,audio09.mp3,지금 지나가는 등산객 30대 남성이 심각한 부상을 입었습니다. 야생 동물에게 다친 것으로 보이는 깊은 상처에서 드랭으로 출혈이 나고 있어요. 의식이 흐릿하고 피가 계속 쏟아 지고 있습니다. 저희 위치는 제주도 우도입니다. 제발 빨리 응급차를 보내주세요.,"* 중요 키워드 : 우도, 등산객, 깊은 상처, 출혈, 의식 흐림, 응급차, * 내용 요약 : 우도 등산객, 다친 깊은 상처 출혈. 의식 흐림, 지속 출혈. 응급차 요청 중."
9,audio11.mp3,안녕하세요. 긴급상황입니다. 지금 10대 남학생이 골절을 당했습니다. 다리가 심하게 부러져있고 통증이 너무 심해 움직일 수 없습니다. 부위가 부풀어 오르고 있습니다. 저희 위치는 태전고등학교 운동장입니다. 제발 빨리 응급차를 보내주세요.,"* 중요 키워드 : 태전고등학교, 운동장, 골절, 통증, 부풀어 오름, 응급차, * 내용 요약 : 태전고등학교 운동장에서 다리 골절, 심한 통증으로 움직일 수 없음. 부위 부풀어 오름. 응급차 요청 중."


In [ ]:
# 모든 열을 출력할 수 있도록 설정
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.max_colwidth', None)  # 모든 열의 너비 제한 해제
pd.set_option('display.expand_frame_repr', True)  # 줄 바꿈 없이 한 줄로 표시

# 데이터프레임 출력
df

,filename,transcript,summary
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.\n,"아빠가 계단에서 넘어져 머리에서 피가 나고 어리작거리는 중. 응급실 방문이 시급합니다. (키워드: 넘어짐, 머리부상, 응급실)"
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈 것 같은데 응급실을 가야 할까요?\n,"화장실에서 미끄러져 꼬리뼈가 아프고 점점 더 아프다고 함. (키워드: 미끄러짐, 엉덩방아, 꼬리뼈, 아픔, 응급실)"
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안 빠져요. 어떻게 해야 할까요? 동생이 너무 힘들어 하네요.\n,"동생이 코에 콩이 걸려 호흡곤란, 응급실 방문이 필요합니다. (키워드: 코에 콩, 호흡곤란, 응급실)"
3,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?\n,"도수 38도, 머리 아픔과 띵함으로 응급 상황 발생. 즉시 병원 방문 권장 (키워드: 열, 머리 아픔, 의식 변화, 응급)"
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리가 어지럽고 속이 매스꺼워요. 어떻게 해야 할까요?\n","머리를 박고 3시간이 지나도 어지러움과 메스꺼움이 계속되고 있음. (키워드: 머리 부딪힘, 어지러움, 메스꺼움, 응급처치 필요)"


### (2) 전국 병원 응급실 정보 수집



#### 1) 인증키 발급

* 인증키 발급 절차
    * 1) data.go.kr 회원가입
    * 2) 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
https://www.data.go.kr/data/15000563/openapi.do 로 이동
    * 3) 활용신청
        * 활용목적 : 기타(개인 학습 용도)
        * 상세 기능선택
            * 응급의료기관 목록정보 조회
            * 응급의료기관 위치정보 조회
            * 응급의료기관 기본정보 조회
    * 4) 인증키 확인
        * 마이페이지 > Open API > 활용신청현황
        * [승인] 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
        * 일반 인증키(Decoding) 이용

#### 2) 데이터 수집

In [ ]:
# path 확인
path

'/content/drive/MyDrive/project6/2/'

In [ ]:
# 응급실 데이터 수집하기

url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire'
serviceKey = '9XqCM96n/WN49HmtWjrLlK357OuTZgjcQlRy2ab5Y9udIquvcqUeZhOixc2XJ4s63kU0h2xDUwbjCaUnbGPLcg==' # 여러분의 일반 인증키(Decoding)

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'format': 'xml'
}

response = requests.get(url, params = params)

# 정상 수행 되었다면 200
print(response)

<Response [200]>


In [ ]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):

    duty_name = item.findtext('dutyName')
    duty_addr = item.findtext('dutyAddr')
    latitude = item.findtext('wgs84Lat') # 위도
    longitude = item.findtext('wgs84Lon') # 경도

    # 필요한 정보 추가

    # 빈 리스트 data에 딕시너리 형태({'칼럼이름':값, ...})로 저장(추가)
    # 딕셔너리 형태로 데이터 추가
    data.append({
        'dutyName': duty_name,
        'dutyAddr': duty_addr,
        'latitude' : latitude,
        'longitude' : longitude

    })

# 'item' 요소의 세부 정보 출력
# for item in root.findall('.//item'):
#     for subitem in item:
#         print(f"{subitem.tag}: {subitem.text}")

# 데이터프레임으로 변환
df = pd.DataFrame(data)



In [ ]:
df

,dutyName,dutyAddr,latitude,longitude
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),35.54823820112527,129.30701143429678
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,35.23602946449906,129.21649161387128
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",37.5089936801167,126.669478649026
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",37.234641294534285,127.21049868474802
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),37.240316373,127.2144914405
...,...,...,...,...
521,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",37.39340413136221,126.92447734066778
522,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",37.35864464913468,126.94735972779895
523,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),35.18541271514478,129.12145899191526
524,흑룡의원,인천광역시 옹진군 백령면 백령로 831,37.959524356,124.6654985764


In [ ]:
# csv 파일로 저장(인덱스 제외)
df.to_csv('./emergency_room.csv', index=False, encoding='utf-8-sig')